In [ ]:
import os
import shutil
import time
import netCDF4 as nc

# test en local: 

In [ ]:
#file_path = "D:/GCxGC_MS/DATA/Centroid/R-L-010-817823-Tedlar.cdf"
#file_path = "D:/GCxGC_MS/DATA/test_adeline/A-F-028-817822-droite-ReCIVA.cdf"
#file_path = "/home/camille/Documents/app/data/input/A-F-028-817822-droite-ReCIVA.cdf"
#A-F-028-817822-droite-ReCIVA.cdf
#file_path = "D:/GCxGC_MS/DATA/test_adeline/J-A-034-751325-Tedlar.cdf"
file_path = "D:/GCxGC_MS/DATA/test_adeline/P-L-007-801838-Tedlar.cdf"


In [ ]:
ds = nc.Dataset(file_path)

In [ ]:
t0 = time.time()
mv = ds["mass_values"][:]
iv = ds["intensity_values"][:]
print("Time to load variables:", time.time() - t0, "s")
print(mv)

# test sur docker

In [ ]:
#file_path = "/app/data/test_adeline/A-F-028-817822-droite-ReCIVA.cdf"
#file_path ="/app/data/test_adeline/J-A-034-751325-Tedlar.cdf"
file_path ="/app/data/test_adeline/P-L-007-801838-Tedlar.cdf"
print(file_path)

In [ ]:
ds = nc.Dataset(file_path)

In [ ]:
t0 = time.time()
print("start")
mv = ds["mass_values"][:]
iv = ds["intensity_values"][:]
print("Time to load variables:", time.time() - t0, "s")

# test copie sur docker avant d analyser

In [ ]:
source_path = file_path = "D:/GCxGC_MS/DATA/624848_0017VE_J0.cdf"
tmp_path = "/app/data/tmp" 
if not os.path.exists(tmp_path):
            os.makedirs(tmp_path)
            print(f"Created output directory: {tmp_path}")
 

In [ ]:
if not os.path.exists(tmp_path):
    print(f"Copying to fast access: {tmp_path}")
    t0 = time.time()
    shutil.copy(source_path, tmp_path)
    print("Time to copy:", time.time() - t0, "s")
else:
    print(f"Using cached copy: {tmp_path}")

In [ ]:
from netCDF4 import Dataset

t0 = time.time()
ds = Dataset(tmp_path)
mass_values = ds["mass_values"][:]
intensity_values = ds["intensity_values"][:]
print("Time to load variables:", time.time() - t0, "s")

# test a partir d un .npy


In [ ]:
import numpy as np
import time

In [ ]:
# file_path = "D:/Dossiers Persos/Adeline/Python-2DGC-Alignment/interface_flask/converted_data/624848_0017VE_J0.npy"
#file_path = "/app/data/P-L-007-801838-Tedlar.npy"
#file_path = "/app/data/A-F-028-817822-droite-ReCIVA.npy"
file_path = "/app/data/J-A-034-751325-Tedlar.npy"
print(file_path)

In [ ]:
t0 = time.time()
print("start")
mass_values = np.load(file_path, allow_pickle=True).item()["mass_values"]
iv = np.load(file_path, allow_pickle=True).item()["intensity_values"]
print("Time to load variables:", time.time() - t0, "s")
print(mass_values)

# test a partir dun .h5

In [1]:
# !pip install h5py
import h5py
import math
import time

In [2]:

file_path = "/home/camille/Documents/app/data/J-A-034-751325-Tedlar.h5"
# file_path = "/app/data/P-L-007-801838-Tedlar.h5"
#file_path = "/app/data/A-F-028-817822-droite-ReCIVA.h5"
# file_path = "/app/data/J-A-034-751325-Tedlar.h5"

In [ ]:
t0 = time.time()
print("start")
with h5py.File(file_path, 'r') as f:
    mass_values = f['mass_values'][:]      # Charge tout
    iv = f['intensity_values'][:]
print("Time to load variables:", time.time() - t0, "s")
print(mass_values)


start
Time to load variables: 2.4191253185272217 s
[ 34.013477  39.02815   39.967106 ... 355.1148   366.1161   429.11288 ]


In [ ]:
t0 = time.time()
print("start")
with h5py.File(file_path, 'r') as f:
        tic_chromato = f['total_intensity'][:]
        timepara = f["scan_acquisition_time"][np.abs(f["point_count"])
                                              < np.iinfo(np.int32).max]
        
        mv = f["mass_values"][:]
        iv = f["intensity_values"][:]
        range_min = math.ceil(f["mass_range_min"][:].min())
        range_max = math.floor(f["mass_range_max"][:].max())
        start_time = f['scan_acquisition_time'][0] / 60
        end_time = f['scan_acquisition_time'][-1] / 60
print("Time to load variables:", time.time() - t0, "s")
print(mass_values)


In [20]:
import math
import h5py
import numpy as np
import netCDF4 as nc

In [ ]:
file_path_h5 = "/home/camille/Documents/app/data/J-A-034-751325-Tedlar.h5"
file_path_cdf = "/home/camille/Documents/app/data/J-A-034-751325-Tedlar.cdf"
mod_time = 1.25



In [47]:
def read_chroma_h5(filename, mod_time, max_val=None):
    if not filename.endswith(".h5"):
        raise ValueError("The file must be a .h5")
    with h5py.File(filename, 'r') as f:
        tic_chromato = f['total_intensity'][:]
        
        # abs_point_count = np.abs(f["point_count"][:])
        # print("abs_point_count", abs_point_count)
        # abs_point_count = np.abs(f["point_count"])
        # print("abs_point_count", abs_point_count)

        timepara = f["scan_acquisition_time"][np.abs(f["point_count"]) < np.iinfo(np.int32).max]
        # print("scan_acquisition_time", scan_acquisition_time)
        print("timepara", timepara)
        if (max_val):
            mv = f["mass_values"][:max_val]
            iv = f["intensity_values"][:max_val]
            
        else:
            mv = f["mass_values"][:]
            print("mv", mv)
            iv = f["intensity_values"][:]
            print("iv", iv)
        range_min = math.ceil(f["mass_range_min"][:].min())
        print("range_min", range_min)
        range_max = math.floor(f["mass_range_max"][:].max())
        print("range_max", range_max)
        start_time = f['scan_acquisition_time'][0] / 60
        print("start_time", start_time)
        end_time = f['scan_acquisition_time'][-1] / 60
        print("end_time", end_time)

    # timepara = scan_acquisition_time[abs_point_count <
    #                                  np.iinfo(np.int32).max]
    # print("timepara", timepara)
    # taux d'échantillonnage : le nombre d'échantillons (points) par unité de temps (par exemple, en Hz).
    sam_rate = 1 / np.mean(timepara[1:] - timepara[:-1])
    l1 = math.floor(sam_rate * mod_time)
    l2 = math.floor(len(tic_chromato) / l1)
    tic_chromato = np.reshape(tic_chromato[:l1*l2], (l2, l1))
    print("l1", l1)
    print("l2", l2)
    print("tic_chromato shape", tic_chromato.shape)
    print("tic_chromato", tic_chromato)

    return (tic_chromato, (start_time, end_time),
            (l1, l2, mv, iv, range_min, range_max))

In [48]:
def read_chroma_cdf(filename, mod_time, max_val=None):

    if not filename.endswith(".cdf"):
        raise ValueError("The file must be a .cdf")

    ds = nc.Dataset(filename, encoding="latin-1")
    tic_chromato = ds['total_intensity']
    abs_point_count = np.abs(ds["point_count"])
    print("abs_point_count", abs_point_count)
    # scan_acquisition_time = ds["scan_acquisition_time"]
    Timepara = ds["scan_acquisition_time"][abs_point_count < np.iinfo(np.int32).max]
    # print("scan_acquisition_time", scan_acquisition_time)
    print("Timepara", Timepara)
    
    # taux d'échantillonnage : le nombre d'échantillons (points) par unité de temps (par exemple, en Hz).
    sam_rate = 1 / np.mean(Timepara[1:] - Timepara[:-1])
    l1 = math.floor(sam_rate * mod_time)
    l2 = math.floor(len(tic_chromato) / l1)

    if (max_val):
        mv = ds["mass_values"][:max_val]
        iv = ds["intensity_values"][:max_val]
    else:
        mv = ds["mass_values"][:]
        print("mv", mv)
        iv = ds["intensity_values"][:]
        print("iv", iv)
    range_min = math.ceil(ds["mass_range_min"][:].min())
    print("range_min", range_min)
    range_max = math.floor(ds["mass_range_max"][:].max())
    print("range_max", range_max)

    tic_chromato = np.reshape(tic_chromato[:l1*l2], (l2, l1))
    print("tic_chromato", tic_chromato)

    start_time = ds['scan_acquisition_time'][0] / 60
    print("start_time", start_time)
    end_time = ds['scan_acquisition_time'][-1] / 60
    print("end_time", end_time)
    print("l1", l1)
    print("l2", l2)
    print("tic_chromato shape", tic_chromato.shape)

    return (tic_chromato, (start_time, end_time),
            (l1, l2, mv, iv, range_min, range_max))


In [49]:
read_chroma_h5(file_path_h5, mod_time, max_val=1000)


timepara [  60.000004   60.005005   60.010002 ... 2759.9873   2759.9922
 2759.9973  ]
range_min 35
range_max 451
start_time 1.0000000635782877
end_time 45.99995524088542
l1 250
l2 2160
tic_chromato shape (2160, 250)
tic_chromato [[3726556.5 3763377.5 3760435.2 ... 3680304.2 3726014.8 3730043.2]
 [3731337.2 3730396.8 3762144.2 ... 3702791.2 3658083.5 3653653.5]
 [3658037.2 3654228.8 3649434.8 ... 3645484.2 3685671.  3683639.2]
 ...
 [5290216.5 5289907.  5288827.  ... 5112671.  5105412.  5092715.5]
 [5088967.5 5085728.5 5086372.5 ... 5132813.5 5144288.  5152023. ]
 [5150250.5 5147483.  5138092.5 ... 7816653.5 7734780.5 7577112. ]]


(array([[3726556.5, 3763377.5, 3760435.2, ..., 3680304.2, 3726014.8,
         3730043.2],
        [3731337.2, 3730396.8, 3762144.2, ..., 3702791.2, 3658083.5,
         3653653.5],
        [3658037.2, 3654228.8, 3649434.8, ..., 3645484.2, 3685671. ,
         3683639.2],
        ...,
        [5290216.5, 5289907. , 5288827. , ..., 5112671. , 5105412. ,
         5092715.5],
        [5088967.5, 5085728.5, 5086372.5, ..., 5132813.5, 5144288. ,
         5152023. ],
        [5150250.5, 5147483. , 5138092.5, ..., 7816653.5, 7734780.5,
         7577112. ]], dtype=float32),
 (1.0000000635782877, 45.99995524088542),
 (250,
  2160,
  array([ 34.013477,  39.02815 ,  39.967106,  41.040943,  42.023464,
          43.02658 ,  43.995834,  45.037445,  55.0406  ,  56.048977,
          57.039623,  58.038258,  59.046158,  60.02924 ,  61.01165 ,
          63.02126 ,  64.0114  ,  65.037056,  66.03755 ,  67.051674,
          68.035095,  69.01254 ,  70.04669 ,  71.04246 ,  72.05407 ,
          73.03497 ,  74.020

In [50]:
read_chroma_cdf(file_path_cdf, mod_time, max_val=1000)

abs_point_count [211 207 201 ... 307 305 307]
Timepara [  60.0000035    60.00500318   60.01000286 ... 2759.98719326 2759.99219294
 2759.99719262]
range_min 35
range_max 451
tic_chromato [[3726556.3925382  3763377.42105451 3760435.1504515  ... 3680304.32976973
  3726014.76919771 3730043.18534159]
 [3731337.31762064 3730396.8617772  3762144.14178141 ... 3702791.22179924
  3658083.43703575 3653653.55829709]
 [3658037.25261709 3654228.67503567 3649434.78102048 ... 3645484.21811937
  3685670.97345216 3683639.31071079]
 ...
 [5290216.71428742 5289906.88106581 5288826.87922381 ... 5112670.94316863
  5105411.85492306 5092715.36123008]
 [5088967.41169193 5085728.33040776 5086372.35005908 ... 5132813.52877956
  5144287.90337346 5152022.8396651 ]
 [5150250.41493393 5147483.03114361 5138092.54772684 ... 7816653.30413991
  7734780.51154945 7577111.78408845]]
start_time 1.0000000584
end_time 45.999953210399994
l1 250
l2 2160
tic_chromato shape (2160, 250)


(masked_array(
   data=[[3726556.3925382 , 3763377.42105451, 3760435.1504515 , ...,
          3680304.32976973, 3726014.76919771, 3730043.18534159],
         [3731337.31762064, 3730396.8617772 , 3762144.14178141, ...,
          3702791.22179924, 3658083.43703575, 3653653.55829709],
         [3658037.25261709, 3654228.67503567, 3649434.78102048, ...,
          3645484.21811937, 3685670.97345216, 3683639.31071079],
         ...,
         [5290216.71428742, 5289906.88106581, 5288826.87922381, ...,
          5112670.94316863, 5105411.85492306, 5092715.36123008],
         [5088967.41169193, 5085728.33040776, 5086372.35005908, ...,
          5132813.52877956, 5144287.90337346, 5152022.8396651 ],
         [5150250.41493393, 5147483.03114361, 5138092.54772684, ...,
          7816653.30413991, 7734780.51154945, 7577111.78408845]],
   mask=False,
   fill_value=1e+20),
 (1.0000000584, 45.999953210399994),
 (250,
  2160,
  masked_array(data=[ 34.0134788 ,  39.02814844,  39.96710464,  41.04094388,
